#Install dependencies

In [1]:
!pip install tensorflow-addons
!pip install transformers
!pip install --upgrade gensim==3.8.3
!pip install optuna
!pip install -q tf-models-official==2.7.0
!pip install wandb -qqq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.5 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
     |████████████████████████████████| 86 kB 2.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.2 MB 62.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-p

# Import libraries. Mount corpora

In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
import pickle as pk
import pandas as pd
import numpy as np
import optuna
import random, os
import wandb

from transformers import TFAutoModel, AutoTokenizer
from google.colab import drive
from pathlib import Path


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from transformers import AutoModel, AutoTokenizer, TFAutoModel

drive.mount("/content/drive", force_remount=True)
wandb.login()

Mounted at /content/drive


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [68]:
FB_RO_OFFENSE_PATH = Path('/content/drive/MyDrive/Licenta_Busuioc_Gabriel-Razvan_2022/corpora/FB-RO-Offense')
RO_OFFENSE_PATH = Path('/content/drive/MyDrive/Licenta_Busuioc_Gabriel-Razvan_2022/corpora/RO-Offense')

df_trainRO = pd.read_csv(RO_OFFENSE_PATH / 'train_internal.csv')
df_testRO = pd.read_csv(RO_OFFENSE_PATH / 'test.csv')
df_validateRO = pd.read_csv(RO_OFFENSE_PATH / 'validation_internal.csv')
df_trainFBRO = pd.read_csv(FB_RO_OFFENSE_PATH / 'train_internal.csv')
df_testFBRO = pd.read_csv(FB_RO_OFFENSE_PATH / 'test.csv')
df_validateFBRO = pd.read_csv(FB_RO_OFFENSE_PATH / 'validation_internal.csv')

len(df_trainRO), len(df_validateRO), len(df_testRO), len(df_trainFBRO), len(df_validateFBRO), len(df_testFBRO)

(7962, 1991, 2492, 2851, 713, 891)

In [69]:

labelCodificationMap = {"OTHER":0, "PROFANITY":1, "INSULT":2, "ABUSE":3}

x_trainRO, y_trainRO = df_trainRO.text, df_trainRO.label.map(labelCodificationMap)
x_validateRO, y_validateRO = df_validateRO.text, df_validateRO.label.map(labelCodificationMap)
x_testRO, y_testRO = df_testRO.text, df_testRO.label.map(labelCodificationMap)

x_trainFBRO, y_trainFBRO = df_trainFBRO.text, df_trainFBRO.label.map(labelCodificationMap)
x_validateFBRO, y_validateFBRO = df_validateFBRO.text, df_validateFBRO.label.map(labelCodificationMap)
x_testFBRO, y_testFBRO = df_testFBRO.text, df_testFBRO.label.map(labelCodificationMap)


In [70]:
tokenizer = AutoTokenizer.from_pretrained("readerbench/RoBERT-base")
bert = TFAutoModel.from_pretrained("readerbench/RoBERT-base")

Some layers from the model checkpoint at readerbench/RoBERT-base were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at readerbench/RoBERT-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [71]:
allData = tokenizer(x_trainFBRO.tolist(), return_tensors="np")
allData

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


{'input_ids': array([list([3, 8102, 334, 21, 920, 513, 6747, 4]),
       list([3, 29, 385, 41, 13, 3666, 7238, 513, 5112, 38, 107, 57, 33, 6764, 294, 16504, 470, 2740, 14, 22, 2338, 6, 13714, 41, 4]),
       list([3, 35382, 18110, 96, 116, 771, 21, 903, 1278, 868, 57, 1805, 1805, 1805, 4]),
       ..., list([3, 780, 1567, 23551, 9423, 9, 4]),
       list([3, 2356, 37417, 3258, 136, 4]),
       list([3, 721, 240, 10178, 14, 13139, 81, 3146, 77, 57, 334, 316, 11043, 6, 3983, 269, 294, 5142, 35312, 112, 294, 407, 15, 513, 4])],
      dtype=object), 'token_type_ids': array([list([0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ...,
       list([0, 0, 0, 0, 0, 0, 0]), list([0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
      dtype=object), 'attention_mask': array([list([1, 1, 1, 1, 1, 1, 1, 1]),
     

In [72]:
sentencesTokensLen = []
for inp in allData['input_ids']:
  sentencesTokensLen.append(len(inp))
st_dev = np.std(sentencesTokensLen)
mean = np.mean(sentencesTokensLen)

max_len_tokenizer = int(mean) + int(st_dev)
st_dev, mean, max_len_tokenizer 

(10.596175560436825, 18.34163451420554, 28)

In [73]:
Y_trainFBRO = None
Y_validateFBRO = None
Y_testFBRO = None

X_testFBRO = None
X_trainFBRO = None
X_validateFBRO = None

Y_trainTRANSF = None
Y_validateTRANSF = None
Y_testTRANSF = None

X_testTRANSF = None
X_trainTRANSF  = None
X_validateTRANSF  = None


In [ ]:
sweep_config = {
    "method": "bayes", # try grid or random
    "metric": {
      "name": "FGAccuracy",
      "goal": "maximize"   
    },
    "parameters": {
        "dense_size": {
            "values": [16, 32, 64, 128]
        },
        "learning_rate": {
            "values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]
        },
        "batch_size": {
            "values": [16, 32, 64, 128]
        },
        "tokenizer_max_len": {
            "values":  [10, 15, 20, 25, 30]
        }
    }
}


def buildModel_BERT_FBRO(denseSize, tokenizerMaxLen):
  global Y_trainFBRO
  global Y_validateFBRO
  global Y_testFBRO
  global X_testFBRO
  global X_trainFBRO
  global X_validateFBRO


  X_trainFBRO = tokenizer(x_trainFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)
  X_validateFBRO = tokenizer(x_validateFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)
  X_testFBRO = tokenizer(x_testFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)

  Y_trainFBRO = tf.one_hot(y_trainFBRO, depth=4)
  Y_validateFBRO = tf.one_hot(y_validateFBRO, depth=4)
  Y_testFBRO = tf.one_hot(y_testFBRO, depth=4)

  X_testFBRO = list(X_testFBRO.values())
  X_trainFBRO = list(X_trainFBRO.values())
  X_validateFBRO = list(X_validateFBRO.values())


  token_ids = tf.keras.layers.Input((None, ), dtype=np.int32)
  type_ids = tf.keras.layers.Input((None,), dtype=np.int32)
  attention = tf.keras.layers.Input((None,), dtype=np.int32)
  bert_output = bert.bert(input_ids=token_ids, attention_mask=attention, token_type_ids=type_ids)
  cls_output = bert_output.last_hidden_state[:,0,:]
  # avg_output = tf.keras.layers.GlobalAveragePooling1D()(bert_output.last_hidden_state, mask=attention)
  hidden = tf.keras.layers.Dense(denseSize, activation="tanh")(cls_output)
  output = tf.keras.layers.Dense(4, activation="softmax")(hidden) 
  model = tf.keras.Model(inputs=[token_ids, type_ids, attention], outputs=[output])
  model.layers[3].trainable = False
  # model.summary()


  return model


In [ ]:
sweep_config = {
    "method": "bayes", # try grid or random
    "metric": {
      "name": "FGAccuracy",
      "goal": "maximize"   
    },
    "parameters": {
        "dense_size": {
            "values": [16, 32, 64, 128]
        },
        "learning_rate": {
            "values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]
        },
        "batch_size": {
            "values": [16, 32, 64, 128]
        },
        "tokenizer_max_len": {
            "values": [20, 40, 60, 80, 100, 120, 140]
        }
    }
}

def buildModel_BERT_TRANSF(denseSize, tokenizerMaxLen):

  global Y_trainTRANSF
  global Y_validateTRANSF
  global Y_testTRANSF
  global X_testTRANSF
  global X_trainTRANSF 
  global X_validateTRANSF 

  X_trainTRANSF = tokenizer(x_trainRO.tolist() + x_validateRO.tolist() + x_testRO.tolist() + x_trainFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)
  X_validateTRANSF = tokenizer(x_validateFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)
  X_testTRANSF = tokenizer(x_testFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)

  Y_trainTRANSF = tf.one_hot(pd.concat([y_trainRO, y_validateRO, y_testRO, y_trainFBRO]), depth=4)
  Y_validateTRANSF = tf.one_hot(y_validateFBRO, depth=4)
  Y_testTRANSF = tf.one_hot(y_testFBRO, depth=4)

  X_testTRANSF = list(X_testTRANSF.values())
  X_trainTRANSF = list(X_trainTRANSF.values())
  X_validateTRANSF = list(X_validateTRANSF.values())

  token_ids = tf.keras.layers.Input((None, ), dtype=np.int32)
  type_ids = tf.keras.layers.Input((None,), dtype=np.int32)
  attention = tf.keras.layers.Input((None,), dtype=np.int32)
  bert_output = bert.bert(input_ids=token_ids, attention_mask=attention, token_type_ids=type_ids)
  cls_output = bert_output.last_hidden_state[:,0,:]
  # avg_output = tf.keras.layers.GlobalAveragePooling1D()(bert_output.last_hidden_state, mask=attention)
  hidden = tf.keras.layers.Dense(denseSize, activation="tanh")(cls_output)
  output = tf.keras.layers.Dense(4, activation="softmax")(hidden) 
  model = tf.keras.Model(inputs=[token_ids, type_ids, attention], outputs=[output])
  model.layers[3].trainable = False
  # model.summary()


  return model

In [74]:
sweep_config = {
    "method": "bayes", # try grid or random
    "metric": {
      "name": "FGAccuracy",
      "goal": "maximize"   
    },
    "parameters": {
        "dense_size": {
            "values": [64, 128, 256, 512]
        },
        "learning_rate": {
            "values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]
        },
        "batch_size": {
            "values": [16, 32, 64, 128]
        },
        "tokenizer_max_len": {
            "values":  [20, 40, 60, 80, 100, 120, 140]
        }
    }
}


def buildModel_CNN_TRANSF(denseSize, tokenizerMaxLen):
  global Y_trainTRANSF
  global Y_validateTRANSF
  global Y_testTRANSF
  global X_testTRANSF
  global X_trainTRANSF 
  global X_validateTRANSF 

  X_trainTRANSF = tokenizer(x_trainRO.tolist() + x_validateRO.tolist() + x_testRO.tolist() + x_trainFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)
  X_validateTRANSF = tokenizer(x_validateFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)
  X_testTRANSF = tokenizer(x_testFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)

  Y_trainTRANSF = tf.one_hot(pd.concat([y_trainRO, y_validateRO, y_testRO, y_trainFBRO]), depth=4)
  Y_validateTRANSF = tf.one_hot(y_validateFBRO, depth=4)
  Y_testTRANSF = tf.one_hot(y_testFBRO, depth=4)

  X_testTRANSF = list(X_testTRANSF.values())
  X_trainTRANSF = list(X_trainTRANSF.values())
  X_validateTRANSF = list(X_validateTRANSF.values())

  token_ids = tf.keras.layers.Input((None, ), dtype=np.int32)
  type_ids = tf.keras.layers.Input((None,), dtype=np.int32)
  attention = tf.keras.layers.Input((None,), dtype=np.int32)
  bert_output = bert.bert(input_ids=token_ids, attention_mask=attention, token_type_ids=type_ids)
  cls_output = bert_output.last_hidden_state
  # avg_output = tf.keras.layers.GlobalAveragePooling1D()(bert_output.last_hidden_state, mask=attention)
  # hidden = tf.keras.layers.Dense(32, activation="tanh")(cls_output) # try with 768

  net = tf.keras.layers.Conv1D(32, (2), activation='relu')(cls_output)
  net = tf.keras.layers.Conv1D(64, (2), activation='relu')(net)
  net = tf.keras.layers.GlobalMaxPool1D()(net)
  net = tf.keras.layers.Dense(denseSize, activation="relu")(net)
  net = tf.keras.layers.Dropout(0.1,  seed = 0)(net)
  net = tf.keras.layers.Dense(4, activation="softmax", name='classifier')(net)
  # output = tf.keras.layers.Dense(4, activation="softmax")(cls_output)
  model = tf.keras.Model(inputs=[token_ids, type_ids, attention], outputs=[net])
  model.layers[3].trainable = False
  # model.summary()


  return model

In [56]:
sweep_config = {
    "method": "bayes", # try grid or random
    "metric": {
      "name": "FGAccuracy",
      "goal": "maximize"   
    },
    "parameters": {
        "dense_size": {
            "values": [64, 128, 256, 512]
        },
        "learning_rate": {
            "values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]
        },
        "batch_size": {
            "values": [16, 32, 64, 128]
        },
        "tokenizer_max_len": {
            "values": [10, 15, 20, 25, 30, 35, 40, 45, 50]
        }
    }
}

def buildModel_CNN_FBRO(denseSize, tokenizerMaxLen):
  global Y_trainFBRO
  global Y_validateFBRO
  global Y_testFBRO
  global X_testFBRO
  global X_trainFBRO
  global X_validateFBRO

  X_trainFBRO = tokenizer(x_trainFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)
  X_validateFBRO = tokenizer(x_validateFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)
  X_testFBRO = tokenizer(x_testFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)

  Y_trainFBRO = tf.one_hot(y_trainFBRO, depth=4)
  Y_validateFBRO = tf.one_hot(y_validateFBRO, depth=4)
  Y_testFBRO = tf.one_hot(y_testFBRO, depth=4)

  X_testFBRO = list(X_testFBRO.values())
  X_trainFBRO = list(X_trainFBRO.values())
  X_validateFBRO = list(X_validateFBRO.values())

  token_ids = tf.keras.layers.Input((None, ), dtype=np.int32)
  type_ids = tf.keras.layers.Input((None,), dtype=np.int32)
  attention = tf.keras.layers.Input((None,), dtype=np.int32)
  bert_output = bert.bert(input_ids=token_ids, attention_mask=attention, token_type_ids=type_ids)
  cls_output = bert_output.last_hidden_state
  # avg_output = tf.keras.layers.GlobalAveragePooling1D()(bert_output.last_hidden_state, mask=attention)
  # hidden = tf.keras.layers.Dense(32, activation="tanh")(cls_output) # try with 768

  net = tf.keras.layers.Conv1D(32, (2), activation='relu')(cls_output)
  net = tf.keras.layers.Conv1D(64, (2), activation='relu')(net)
  net = tf.keras.layers.GlobalMaxPool1D()(net)
  net = tf.keras.layers.Dense(denseSize, activation="relu")(net) #!!512
  net = tf.keras.layers.Dropout(0.1, seed = 0)(net)
  net = tf.keras.layers.Dense(4, activation="softmax", name='classifier')(net)
  # output = tf.keras.layers.Dense(4, activation="softmax")(cls_output)
  model = tf.keras.Model(inputs=[token_ids, type_ids, attention], outputs=[net])
  model.layers[3].trainable = False
  # model.summary()


  return model

In [75]:
sweep_id = wandb.sweep(sweep_config, project="OffensiveLanguage")

Create sweep with ID: cucfzgb1
Sweep URL: https://wandb.ai/busu/OffensiveLanguage/sweeps/cucfzgb1


In [76]:


def tf_seed(seed=0):
	os.environ['PYTHONHASHSEED'] = str(seed)
	# For working on GPUs from "TensorFlow Determinism"
	os.environ["TF_DETERMINISTIC_OPS"] = str(seed)
	np.random.seed(seed)
	random.seed(seed)
	tf.random.set_seed(seed)
	from keras import backend as K
	session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
	sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
	K.set_session(sess)

tf_seed(0)

In [77]:
from sklearn import metrics
def getMetrics(dense_size, learning_rate, batch_size, tokenizer_max_len):
  print(dense_size, learning_rate, batch_size, tokenizer_max_len)
  model = buildModel_CNN_TRANSF(dense_size, tokenizer_max_len)
  epochs = 30

  f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
  f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=[tf.metrics.CategoricalAccuracy(), tf.metrics.Precision(), tf.metrics.Recall(), f1_metric_micro, f1_metric_macro])
  

  callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=2,
    mode='auto', baseline=None, restore_best_weights=True)
  
  # history = model.fit(x=X_trainFBRO, y=Y_trainFBRO, validation_data=(X_validateFBRO, Y_validateFBRO), epochs=epochs, batch_size=batch_size, callbacks=[callback], verbose=0)
  # results = model.evaluate(X_testFBRO, Y_testFBRO, verbose=0)

  history = model.fit(x=X_trainTRANSF, y=Y_trainTRANSF, validation_data=(X_validateTRANSF, Y_validateTRANSF), epochs=epochs, batch_size=batch_size, callbacks=[callback], verbose=0)
  results = model.evaluate(X_testTRANSF, Y_testTRANSF, verbose=0)
  
  # y_predFBRO = model.predict(X_testFBRO)
  # y_predCG = pd.Series(tf.argmax(y_predFBRO, 1)).map({0:0, 1:1, 2:1, 3:1})
  # Y_testCG = pd.Series(tf.argmax(Y_testFBRO, 1)).map({0:0, 1:1, 2:1, 3:1})

  y_predTRANSF = model.predict(X_testTRANSF)
  y_predCG = pd.Series(tf.argmax(y_predTRANSF, 1)).map({0:0, 1:1, 2:1, 3:1})
  Y_testCG = pd.Series(tf.argmax(Y_testTRANSF, 1)).map({0:0, 1:1, 2:1, 3:1})

  CGaccuracy = metrics.accuracy_score(Y_testCG, y_predCG)
  CGprecision = metrics.precision_score(Y_testCG, y_predCG)
  CGrecall = metrics.recall_score(Y_testCG, y_predCG)
  CGf1 = metrics.f1_score(Y_testCG, y_predCG)

  # model.save_weights('/content/BERT-CNN_TRANSF_'+ str(format(results[1], ".3f")) + '_' + str(dense_size) + '_' + str(learning_rate) + '_' + str(batch_size) + '/')
  tf.keras.models.save_model(model, '/content/model_'+ str(format(results[1], ".3f")) + '_' + str(dense_size) + '_' + str(tokenizer_max_len) + '_' + str(batch_size) + '/')

  return results + [CGaccuracy, CGprecision, CGrecall, CGf1]

In [ ]:
from sklearn import metrics
def buildAndTrainModel(dense_size, learning_rate, batch_size, tokenizer_max_len):
  print(dense_size, learning_rate, batch_size, tokenizer_max_len)
  model = buildModel_CNN_FBRO(dense_size, tokenizer_max_len)
  epochs = 30

  f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
  f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=[tf.metrics.CategoricalAccuracy(), tf.metrics.Precision(), tf.metrics.Recall(), f1_metric_micro, f1_metric_macro])
  

  callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=2,
    mode='auto', baseline=None, restore_best_weights=True)
  
  history = model.fit(x=X_trainFBRO, y=Y_trainFBRO, validation_data=(X_validateFBRO, Y_validateFBRO), epochs=epochs, batch_size=batch_size, callbacks=[callback], verbose=1)
  results = model.evaluate(X_testFBRO, Y_testFBRO, verbose=1)

  # history = model.fit(x=X_trainTRANSF, y=Y_trainTRANSF, validation_data=(X_validateTRANSF, Y_validateTRANSF), epochs=epochs, batch_size=batch_size, callbacks=[callback], verbose=0)
  # results = model.evaluate(X_testTRANSF, Y_testTRANSF, verbose=0)
  
  y_predFBRO = model.predict(X_testFBRO)
  y_predCG = pd.Series(tf.argmax(y_predFBRO, 1)).map({0:0, 1:1, 2:1, 3:1})
  Y_testCG = pd.Series(tf.argmax(Y_testFBRO, 1)).map({0:0, 1:1, 2:1, 3:1})

  # y_predTRANSF = model.predict(X_testTRANSF)
  # y_predCG = pd.Series(tf.argmax(y_predTRANSF, 1)).map({0:0, 1:1, 2:1, 3:1})
  # Y_testCG = pd.Series(tf.argmax(Y_testTRANSF, 1)).map({0:0, 1:1, 2:1, 3:1})

  CGaccuracy = metrics.accuracy_score(Y_testCG, y_predCG)
  CGprecision = metrics.precision_score(Y_testCG, y_predCG)
  CGrecall = metrics.recall_score(Y_testCG, y_predCG)
  CGf1 = metrics.f1_score(Y_testCG, y_predCG)

  print(results + [CGaccuracy, CGprecision, CGrecall, CGf1])

  return model

In [78]:
def getM():
  config_defaults = {
  "dense_size": 3,
  "learning_rate": 0.1,
  "batch_size": 1,
  "tokenizer_max_len": 117}

  wandb.init(config=config_defaults)  # defaults are over-ridden during the sweep
  config = wandb.config

  metrics = getMetrics(config.dense_size, config.learning_rate, config.batch_size, config.tokenizer_max_len)
  wandb.log({"FGAccuracy": metrics[1], 
             "FGPrecision": metrics[2],
             "FGRecall": metrics[3],
             "FGF1Micro": metrics[4],
             "FGF1Macro": metrics[5],
             "CGAccuracy": metrics[6],
             "CGPrecision": metrics[7],
             "CGRecall": metrics[8],
             "CGF1": metrics[9]})

In [ ]:
wandb.agent(sweep_id, getM, count=50)

wandb: Agent Starting Run: ph7osaur with config:
wandb: 	batch_size: 128
wandb: 	dense_size: 512
wandb: 	learning_rate: 0.0001
wandb: 	tokenizer_max_len: 40
wandb: Currently logged in as: busu. Use `wandb login --relogin` to force relogin


512 0.0001 128 40


In [ ]:

def objective(trial):

    dense_size = trial.suggest_categorical('dense_size', [64, 128, 256, 512])
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]) 
    batch_size = trial.suggest_categorical('per_gpu_batch_size', [16, 32, 64, 128])
    tokenizer_max_len = trial.suggest_categorical('tokenizer_max_len', [10, 15, 20, 25, 30])

    metrics = getMetrics(dense_size, learning_rate, batch_size, tokenizer_max_len)

    return metrics
    
study = optuna.create_study(study_name="FBROstudy-BERT-CNN", directions=['minimize', 'maximize', 'maximize', 'maximize', 'maximize', 'maximize', 'maximize', 'maximize', 'maximize', 'maximize'], storage="sqlite:///FBROstudy-BERT-CNN.db")
study.optimize(objective, n_trials=50)



In [ ]:
from google.colab import files
files.download('/content/FBROstudy-BERT-CNN.db') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Rerun model


In [59]:
weightsPath = '/content/BERT-CNN_TRANSF_0.835_64_0.001_16/' #change this
learning_rate = 1e-3

loaded_model = buildModel_CNN_FBRO(64, 50)
# loaded_model.load_weights(weightsPath)
loaded_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=tf.metrics.CategoricalAccuracy())

results = loaded_model.evaluate(X_testFBRO, Y_testFBRO, verbose=1)
results

28/28 [==============================] - 6s 91ms/step - loss: 1.3752 - categorical_accuracy: 0.2110


[1.375222086906433, 0.21099887788295746]

In [67]:
# loaded_model.save("/")
# !mkdir -p /content/saved_model + "/ceva"
tf.keras.models.save_model(loaded_model, "/content/saved_model2/")

INFO:tensorflow:Assets written to: /content/saved_model2/assets


INFO:tensorflow:Assets written to: /content/saved_model2/assets


In [63]:
m = tf.keras.models.load_model("/content/saved_model")

In [64]:
results = m.evaluate(X_testFBRO, Y_testFBRO, verbose=1)
results

28/28 [==============================] - 6s 91ms/step - loss: 1.3752 - categorical_accuracy: 0.2110


[1.375222086906433, 0.21099887788295746]

# Rebuild certain model

In [65]:
# weightsPath = '' #change this
# seed = 0 #change this
# batch_size = 32 #change this
# learing_rate = 1e-3 # change this
# epochs = 30

for i in range(5):
  # tf_seed(0)
  print(getMetrics(64, 0.001, 16, 50))

# def buildAndTrainModel(dense_size, learning_rate, batch_size, tokenizer_max_len):
# buildModel = buildAndTrainModel(64, 0.001, 16, 50)

# metrics = getMetrics(config.dense_size, config.learning_rate, config.batch_size, config.tokenizer_max_len)
# loaded_model = buildModel()
# loaded_model.load_weights(weightsPath)

# loaded_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#                 loss=tf.keras.losses.CategoricalCrossentropy(),
#                 metrics=tf.metrics.CategoricalAccuracy())
  
# loaded_callback = tf.keras.callbacks.EarlyStopping(
#   monitor='val_loss', min_delta=0, patience=3, verbose=2,
#   mode='auto', baseline=None, restore_best_weights=True)

# history = loaded_model.fit(x=X_trainFBRO, y=Y_trainFBRO, validation_data=(X_validateFBRO, Y_validateFBRO), epochs=epochs, batch_size=batch_size, callbacks=[loaded_callback], verbose=1)
# results = loaded_model.evaluate(X_testFBRO, Y_testFBRO, verbose=0)
# results

Restoring model weights from the end of the best epoch: 4.
Epoch 7: early stopping
[0.5233478546142578, 0.8215488195419312, 0.8498817682266235, 0.8069584965705872, 0.8278641104698181, 0.6245710849761963, 0.8832772166105499, 0.9207920792079208, 0.7771587743732591, 0.8429003021148037]
64 0.001 16 50


KeyboardInterrupt: ignored